# Scrapping the main table

We will get the players' stats and salaries by going through this notebook.

At first, we need to set all needed filters and conditions on players. Then we will use the resulting URL to get data.

For example, I will scrap all the players through last 5 seasons. 

As I need data season-by-season I am going to start from the last season to the earliest I need.

This link will allow me to download data for the 2020-2021 season with all parameters specified.

Let's do some coding now.

In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging

In [34]:
req = requests.get("https://www.capfriendly.com/browse/active/2022?stats-season=2022&display=birthday,country,weightkg,heightcm,draft,slide-candidate,signing-status,expiry-year,performance-bonus,caphit-percent,aav,length,minors-salary,base-salary,skater-individual-advanced-stats,skater-on-ice-advanced-stats,goalie-advanced-stats,type,signing-age,signing-date,arbitration,extension&limits=gp-5-90")
soup = BeautifulSoup(req.content)


In [53]:
info_about_lists = soup.find_all("a", {"class": "whi pagin_r"})
last_list_num = int(info_about_lists[-1]["data-val"])  # take the last number of page from date-val

In [58]:
dfs_array = []
for page_num in range(1, last_list_num + 1):

    time.sleep(2)  # let the page download the results

    url = f"https://www.capfriendly.com/browse/active/2022?stats-season=2022&display=birthday,country,weightkg,heightcm,draft,slide-candidate,signing-status,expiry-year,performance-bonus,caphit-percent,aav,length,minors-salary,base-salary,skater-individual-advanced-stats,skater-on-ice-advanced-stats,goalie-advanced-stats,type,signing-age,signing-date,arbitration,extension&limits=gp-5-90&pg={page_num}"
    df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]

    dfs_array.append(df)

In [63]:
result_df = pd.concat(dfs_array)

In [62]:
result_df.shape

(978, 65)

In [64]:
result_df.head()

,TEAM,AGE,DATE OF BIRTH,COUNTRY,WEIGHT,HEIGHT,POS,HANDED,DRAFTED,SLIDE CAND.,...,SIGNING,EXPIRY,EXP. YEAR,CAP HIT,CAP HIT %,AAV,SALARY,BASE SALARY,MINORS,P.BONUS
PLAYER,,,,,,,,,,,,,,,,,,,,,
1. Connor McDavid,EDM,24,"Jan. 13, 1997",Canada,88,185,C,Left,1 - Round 1 - 2015 (EDM),NaN,...,RFA,UFA,2026,"$12,500,000",16.7%,"$12,500,000","$13,000,000","$1,000,000","$13,000,000",$0
2. Artemi Panarin,NYR,29,"Oct. 30, 1991",Russia,77,180,LW,Right,NaN,NaN,...,UFA,UFA,2026,"$11,642,857",14.3%,"$11,642,857","$13,000,000","$1,000,000","$13,000,000",$0
3. Auston Matthews,TOR,23,"Sep. 17, 1997",United States,100,191,C,Left,1 - Round 1 - 2016 (TOR),NaN,...,RFA,UFA,2024,"$11,640,250",14.6%,"$11,640,250","$10,470,000","$750,000","$10,470,000",$0
4. Erik Karlsson,SJS,31,"May 31, 1990",Sweden,86,183,RD,Right,15 - Round 1 - 2008 (OTT),NaN,...,UFA,UFA,2027,"$11,500,000",14.5%,"$11,500,000","$14,500,000","$5,500,000","$14,500,000",$0
5. John Tavares,TOR,30,"Sep. 20, 1990",Canada,98,185,C,Left,1 - Round 1 - 2009 (NYI),NaN,...,UFA,UFA,2025,"$11,000,000",13.8%,"$11,000,000","$9,350,000","$910,000","$9,350,000",$0


In [70]:
seasons_df = []

for year in range (2016, 2023):

    url_start = f"https://www.capfriendly.com/browse/active/{year}?stats-season={year}&display=birthday,country,weightkg,heightcm,draft,slide-candidate,signing-status,expiry-year,performance-bonus,caphit-percent,aav,length,minors-salary,base-salary,skater-individual-advanced-stats,skater-on-ice-advanced-stats,goalie-advanced-stats,type,signing-age,signing-date,arbitration,extension&limits=gp-5-90"

    req = requests.get(url_start)

    soup = BeautifulSoup(req.content)

    info_about_lists = soup.find_all("a", {"class": "whi pagin_r"})
    last_list_num = int(info_about_lists[-1]["data-val"])  # take the last number of page from date-val

    logging.info(f"Start season {year} scapring")
    time.sleep(5)

    dfs_array = []

    for page_num in range(1, last_list_num + 1):

        logging.info(f"Start season {year} scapring page {page_num}")

        time.sleep(2)  # let the page download the results

        url = url_start + f"&pg={page_num}"
        df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]

    dfs_array.append(df)

    logging.info(f"Finish season {year} scapring page {page_num}")

    one_season_df = pd.concat(dfs_array)

    seasons_df.append(one_season_df)

    logging.info(f"Finish season {year} scapring")




In [71]:
total_df = pd.concat(seasons_df)

In [72]:
total_df.to_csv("my_dataset.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fb97efa2-d417-46e7-813c-0372ce0dd7f6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>